In [ ]:
!pip install easyocr

In [ ]:
! pip install sqlite-ulid sqlite-jsonschema sqlite-fastrand sqlite-xsv

**download folder from:-**
https://drive.google.com/drive/folders/1R94Hm8JkgteKLMnngm4QzsXvvPIdWUdV?usp=sharing

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

#download link given above and save in your drive

/content/gdrive/MyDrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/AI project

ln: failed to create symbolic link '/mydrive/My Drive': File exists
/content/gdrive/My Drive/AI project


In [ ]:
import cv2
import numpy as np
import os.path
import easyocr
from matplotlib import pyplot as plt
import sqlite3
import datetime

In [ ]:
classesFile = "./names.txt"
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
modelConfiguration = './yolov4-custom.cfg'
modelWeights = './yolov4-custom_best.weights'

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
def remove_special_chars(s):
    # make a array of char you want to remove
    special_chars = ['.', ',', '-', '_', '\\', '/',' ',';','[',']',':',"'"]
    for char in special_chars:
      # replce the char with ''
        s = s.replace(char, '')
    return s

def most_frequent_string(arr):
    # Initialize a dictionary to count the frequency of each string
    freq = {}

    # Loop through the array and count the frequency of each string
    for string in arr:
        if string in freq:
            freq[string] += 1
        else:
            freq[string] = 1

    # Find the string with the highest frequency
    max_freq = 0
    most_freq_string = ""
    for string in freq:
        if freq[string] > max_freq:
            max_freq = freq[string]
            most_freq_string = string

    # Return the most frequent string
    return most_freq_string


In [ ]:
def findObjects(outputs, img,end):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w, h = int(det[2]*wT), int(det[3]*hT)
                x, y = int((det[0]*wT) - w/2), int((det[1]*hT) - h/2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))
    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)

    for i in indices:

        i = i
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        # print(x,y,w,h)

        crop_img = img[y:y+h, x:x+w]
        # print(len(crop_img))
        if (x<=0 or y<=0 or w<=0 or h<=0):
          print('crop_img is empty')
        else:
          gray_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
          reader = easyocr.Reader(['en'])
          result = reader.readtext(gray_img)
          if(result[0][1]==[]):
            print('empty')
            return result_rmsp,4
          else:
            result_rmsp=remove_special_chars(result[0][1])
            end=end+1
            # print(end)
            print(result_rmsp)
            return result_rmsp,end

In [ ]:
whT = 320
confThreshold = 0.7
nmsThreshold = 0.3
task = 0
resultarr=[]

videopath = "./test_videos/v" + str(4) + ".mp4"
cap = cv2.VideoCapture(videopath)

while True:
    success, img = cap.read()
    blob = cv2.dnn.blobFromImage(
        img, 1/255, (whT, whT), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layerNames = net.getLayerNames()

    outputNames = [layerNames[i-1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)
    result,task = findObjects(outputs, img,task)
    resultarr.append(result)
    if(task==4):
      break
cap.release()

MHO1TC0502
MH01TC0502
MH01TC0502
MhO1TC0502


In [ ]:
print(resultarr)
final_detected = most_frequent_string(resultarr)
current_time = datetime.datetime.now().strftime("%H:%M:%S")
final_detected=final_detected.upper()
print(final_detected)

['MHO1TC0502', 'MH01TC0502', 'MH01TC0502', 'MhO1TC0502']
MH01TC0502
